In [1]:
import psycopg2
import os
import traceback
import pandas as pd
import urllib.request
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(funcName)s:%(levelname)s:(massage)s')

In [6]:
postgres_host = 'postgres'
postgres_database = 'processing'
postgres_user = os.environ.get('POSTGRES_USER')
postgres_password = os.environ.get('POSTGRES_PASSWORD')
postgres_port = '5432'
dest_folder = './data'

In [7]:
url = 'https://raw.githubusercontent.com/xploiterx/datasets/master/Proyect-0/CSV/Churn_Modelling-1.csv'
destination_path = f'{dest_folder}/Churn_Modelling-1.csv'

In [8]:
conn = psycopg2.connect(
    host = postgres_host,
    database = postgres_database,
    user = postgres_user,
    password = postgres_password,
    port = postgres_port
)

In [9]:
try:
    conn = psycopg2.connect(
    host = postgres_host,
    database = postgres_database,
    user = postgres_user,
    password = postgres_password,
    port = postgres_port
    )
    cur = conn.cursor()
    logging.info("La conexion a postgres es correcta")

except Exception as e:
    traceback.print_exc()
    logging.error("No se pudo crear la conexion")

2024-08-25 20:16:09,191:<module>:INFO:(massage)s


In [10]:
def download_file_from_url(url: str, dest: str):
    if not os.path.exists(str(dest)):
        os.makedirs(str(dest))

    try:
        urllib.request.urlretrieve(url,destination_path)
        logging.info('Archivo descargado exitosamente al directorio de trabajo')
    except Exception as e:
        logging.error(f'Error al descargar el archivo CSV debido a: {e}')
        traceback.print_exc()

In [11]:
def create_postgres_table():
    """
    Crea la tabla de Postgres con un esquema deseado
    """
    try:
        cur.execute("""CREATE TABLE IF NOT EXISTS churn_modelling (RowNumber INTEGER PRIMARY KEY, CustomerId INTEGER, 
        Surname VARCHAR(50), CreditScore INTEGER, Geography VARCHAR(50), Gender VARCHAR(20), Age INTEGER, 
        Tenure INTEGER, Balance FLOAT, NumOfProducts INTEGER, HasCrCard INTEGER, IsActiveMember INTEGER, EstimatedSalary FLOAT, Exited INTEGER)""")
        
        logging.info(' Nueva tabla churn_modelling creada exitosamente en el servidor de postgres')
    except:
        logging.warning(' Verifica si la tabla churn_modelling existe')

In [12]:
def write_to_postgres():
    """
    Crea el dataframe y escribe en la tabla de Postgres si aún no existe
    """
    df = pd.read_csv(f'{dest_folder}/Churn_Modelling-1.csv')
    inserted_row_count = 0

    for _, row in df.iterrows():
        count_query = f"""SELECT COUNT(*) FROM churn_modelling WHERE RowNumber = {row['RowNumber']}"""
        cur.execute(count_query)
        result = cur.fetchone()
        
        if result[0] == 0:
            inserted_row_count += 1
            cur.execute("""INSERT INTO churn_modelling (RowNumber, CustomerId, Surname, CreditScore, Geography, Gender, Age, 
            Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s)""", 
            (int(row[0]), int(row[1]), str(row[2]), int(row[3]), str(row[4]), str(row[5]), int(row[6]), int(row[7]), float(row[8]), int(row[9]), int(row[10]), int(row[11]), float(row[12]), int(row[13])))

    logging.info(f' {inserted_row_count} filas del archivo CSV insertadas en la tabla churn_modelling con éxito')

In [13]:
if __name__ == '__main__':
    download_file_from_url(url, dest_folder)
    create_postgres_table()
    write_to_postgres()
    conn.commit()
    cur.close()
    conn.close()

2024-08-25 20:23:32,102:download_file_from_url:INFO:(massage)s
2024-08-25 20:23:32,189:create_postgres_table:INFO:(massage)s
2024-08-25 20:23:52,232:write_to_postgres:INFO:(massage)s
